<a href="https://colab.research.google.com/github/GustavoGatti/BreastCancerSiameseNeuralNetwork/blob/main/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Teste da implementação da redes neurais siameses convolucional





In [ ]:
!pip install tensorflow-gpu==2.2.0-rc0

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization, Input, Lambda, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
from keras.regularizers import l2

from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from numpy import mean
from numpy import std
from numpy import ndarray
import numpy.random as rng
import random

In [ ]:
from keras import backend as K
import keras as KER

In [ ]:
from keras.layers import merge

In [ ]:
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

In [ ]:
def cosine(vector):
  x, y = vector
  dotx = K.sum(x*x, axis=1,keepdims=True)
  doty = K.sum(y*y, axis=1,keepdims=True)
  dotxy = K.sum(x*y, axis=1,keepdims=True)
  cos = (dotxy/(K.sqrt(dotx) * K.sqrt(doty)))
  return cos

In [ ]:
def eucl_dist_output_shape(shapes):
  shape1, shape2 = shapes
  return shape1[0], 1

In [ ]:
def contrastive_loss(y_true,y_pred):
  return K.mean(y_true*(2*K.exp(-(1/K.square(y_pred-1.2)))) + (1 - y_true)*(2*K.exp(-(1/K.square(y_pred+1.2)))))

In [ ]:
def logistic_loss(y_true,y_pred):
  return K.mean(K.log(1 * K.exp(-1*y_pred*y_true)))

In [ ]:
def compute_accuracy(y_true,y_pred):
  classified = K.round(y_pred)
  return K.mean(K.equal(classified, y_true))

In [ ]:
def compute_final_accuracy(predictions, labels, print_prediction = False):
  if print_prediction:
    print(predictions)

  classified = np.round(predictions.ravel())
  return np.mean(classified == labels)

In [ ]:
def difference(vector):
  x, y = vector
  return x-y

In [ ]:
def cos_distance(vector):
  return 1-cosine(vector)

In [ ]:
def distance(cos):
  return 1 - K.abs(cos)

In [ ]:
initialize_weights = KER.initializers.RandomNormal(mean=0.0, stddev=0.51, seed=50001)
initialize_bias = KER.initializers.RandomNormal(mean=0.0, stddev=0.1, seed=1221)

In [ ]:
def criando_rede(input_shape):


  model = Sequential()
  model.add(Conv1D(filters=32, kernel_size=2, activation = 'relu', input_shape = input_shape, kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4)))
  model.add(BatchNormalization())

  model.add(Conv1D(filters=64, kernel_size=2, activation = 'relu',kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))
  model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(64, activation = 'relu',kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4)))

  model.add(Dense(1, activation = 'sigmoid', kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
  '''
  model = Sequential()
  model.add(keras.layers.Reshape(input_shape = (9, 1), target_shape=(9,1)))
  model.add(Conv1D(filters=32, kernel_size=2, activation = 'relu', input_shape = (9,1)))
  model.add(BatchNormalization())
  
  model.add(Conv1D(filters=64, kernel_size=2, activation = 'relu'))
  model.add(BatchNormalization())
  
  model.add(Dense(64, activation = 'relu'))
  model.add(Flatten())
  model.add(Dense(1, activation = 'sigmoid'))
  '''

  return model

In [ ]:
'''
def execute(epochs=50):
  np.random.seed(100)
  random.seed(100)

  #creat dataset

  input_dim = (9,1)

  rede_base = criando_rede(input_dim)
  input_left = Input(input_dim)
  input_right = Input(input_dim)

  left_side = rede_base(input_left)
  right_side = rede_base(input_right)

  both = merge([left_side, right_side], mode=cosine, output_shape=eucl_dist_output_shape)

  model = Model(input=[input_left, input_right], output=both)

  loss = contrastive_loss
  optimizer = Adam(0.00005)
  model.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=["acurracy"])
  hist = model.fit(train_x, train_y, epochs=50, validation_data=(test_x, test_y))
'''

'\ndef execute(epochs=50):\n  np.random.seed(100)\n  random.seed(100)\n\n  #creat dataset\n\n  input_dim = (9,1)\n\n  rede_base = criando_rede(input_dim)\n  input_left = Input(input_dim)\n  input_right = Input(input_dim)\n\n  left_side = rede_base(input_left)\n  right_side = rede_base(input_right)\n\n  both = merge([left_side, right_side], mode=cosine, output_shape=eucl_dist_output_shape)\n\n  model = Model(input=[input_left, input_right], output=both)\n\n  loss = contrastive_loss\n  optimizer = Adam(0.00005)\n  model.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=["acurracy"])\n  hist = model.fit(train_x, train_y, epochs=50, validation_data=(test_x, test_y))\n'

In [ ]:
def Transformacao(dataset):

  novo_data = pd.DataFrame(data=dataset)
  novo_data.drop('Classification',inplace=True,axis=1)  
  data_x = novo_data 
  data_y = dataset.Classification
  x_train = np.asarray(data_x)
  y_train = np.asarray(data_y)
  print(x_train)
  test_x = []
  test_y = []
  for i in range(116):
    if(i<7):
      test_x.append(x_train[i])
      test_y.append(y_train[i])
    elif(i>99):
      test_x.append(x_train[i])
      test_y.append(y_train[i])


  test_x = np.asarray(test_x)
  test_y = np.asarray(test_y)
  print(test_x.shape)
  print(test_y.shape)

  scaler = StandardScaler()
  test_x = scaler.fit_transform(test_x)
  x_train = scaler.fit_transform(x_train)
  x_train = x_train.reshape(116,9,1)
  test_x = test_x.reshape(23,9,1)

  return test_x, test_x, test_y, test_y, scaler

In [ ]:
def Montando_data_set(x_train,x_train1,y_train,y_train1):
  target = []
  trainx = []
  trainx1 =[]
  print(y_train)
  print(y_train1)
  print(len(x_train))
  print(len(x_train1))
  for i in range(23):
    for j in range(23):
      trainx.append(x_train[i])
      trainx1.append(x_train1[j])
      if(y_train[i] == y_train1[j]):
        target.append(1)
      else:
        target.append(0)
  trainx=np.asarray(trainx)
  trainx1=np.asarray(trainx1)
  xtreino = [trainx,trainx1]
  target = np.asarray(target)

  return xtreino,target

In [ ]:
def Transformacao_teste(dataset):

  novo_data = pd.DataFrame(data=dataset)
  novo_data.drop('Classification',inplace=True,axis=1)  
  data_x = novo_data 
  data_y = dataset.Classification
  x_train = np.asarray(data_x)
  y_train = np.asarray(data_y)
  print(x_train)
  test_x = []
  test_y = []
  y = 0
  for i in range(116):
    if((i>7) & (i<15)):
      test_x.append(x_train[i])
      test_y.append(y_train[i])
      y = y+1
    elif((i>85) & (i<95) ):
      test_x.append(x_train[i])
      test_y.append(y_train[i])
      y = y+1

  print(y)
  test_x = np.asarray(test_x)
  test_y = np.asarray(test_y)

  scaler = StandardScaler()
  test_x = scaler.fit_transform(test_x)
  print(test_x.shape)
  test_x = test_x.reshape(16,9,1)

  return test_x, test_x, test_y, test_y

In [ ]:
'''
cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
x_train1, x_train2, y_train1, y_train2 = _Dataset(cancer)
x_train, y_train = Montando_data_set(x_train1,x_train2, y_train1, y_train2)
'''

"\ncancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')\nx_train1, x_train2, y_train1, y_train2 = _Dataset(cancer)\nx_train, y_train = Montando_data_set(x_train1,x_train2, y_train1, y_train2)\n"

In [ ]:
def Montando_data_set_test(x_train,x_train1,y_train,y_train1):
  target = []
  trainx = []
  trainx1 =[]
  print(y_train)
  print(y_train1)
  print(len(x_train))
  print(len(x_train1))
  for i in range(16):
    for j in range(16):
      trainx.append(x_train[i])
      trainx1.append(x_train1[j])
      if(y_train[i] == y_train1[j]):
        target.append(1)
      else:
        target.append(0)
  trainx=np.asarray(trainx)
  trainx1=np.asarray(trainx1)
  xtreino = [trainx,trainx1]
  target = np.asarray(target)

  return xtreino,target

In [ ]:
cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
x, x1, y, y1, scaler = Transformacao(cancer)
x_train, y_train = Montando_data_set(x,x1,y,y1)

[[ 48.          23.5         70.         ...   9.7024       7.99585
  417.114     ]
 [ 83.          20.69049454  92.         ...   5.429285     4.06405
  468.786     ]
 [ 82.          23.12467037  91.         ...  22.43204      9.27715
  554.697     ]
 ...
 [ 65.          32.05        97.         ...  22.54        10.33
  314.05      ]
 [ 72.          25.59        82.         ...  33.75         3.27
  392.46      ]
 [ 86.          27.18       138.         ...  14.11         4.35
   90.09      ]]
(23, 9)
(23,)
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
23
23


In [ ]:
cancer = pd.read_csv('/content/drive/My Drive/Colab Notebooks/dataR234.csv')
x12, x123, y12, y123 = Transformacao_teste(cancer)
x_test, y_test = Montando_data_set_test(x12,x123,y12,y123)

In [ ]:
def get_classifier_model(input_shape):
    """
        Model architecture
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = criando_rede(input_shape)


    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:KER.backend.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [ ]:
ip_shape = (9,1)
model = get_classifier_model(ip_shape)
epochs = 700
optimizer = Adam(0.0005)
model.compile(loss="binary_crossentropy",optimizer=optimizer, metrics=['accuracy'])

ValueError: ignored

In [ ]:
model.summary()

In [ ]:
hist = model.fit(x_train, y_train, epochs=epochs,validation_data=(x_test,y_test), verbose=1)

In [ ]:
def plot_learningCurve(history, epochs):

  #Plot Training e validation accuracy values
  epoch_range = range(1, epochs+1)
  plt.plot(epoch_range, history.history['accuracy'])
  plt.plot(epoch_range, history.history['val_accuracy'])
  plt.title('Model Accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

  #Plot training e validation accuracy loss values
  plt.plot(epoch_range, history.history['loss'])
  plt.plot(epoch_range, history.history['val_loss'])
  plt.title('Model Loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()

In [ ]:
plot_learningCurve(hist, epochs)

In [ ]:
results = model.evaluate(x_test, y_test, batch_size=128)
print('test loss, test acc:', results)

Predict Modelo


In [ ]:
def predict(model, x_aux):

    prediction = model.predict(x_aux)
    print(prediction)

    prediction_index = np.argmin(prediction, axis=1)
    print(" Prediction index: {}".format( prediction_index))

In [ ]:
#48	,23.5,	70,	2.707	,0.467408666666667,	8.8071,	9.7024,	7.99585	,417.114 controle

#58,	29.1545189504373	,139,	16.582,	5.68541506666667,	22.8884	,10.26266,	13.97399	,923.886 cancer

#73,	37.109375,	134,	5.636,	1.86288586666667,	41.4064	,3.335665,	6.89235,	788.902 cancer

#76,	23.8	,118,	6.47,	1.88320133333333,	4.311,	13.25132	,5.1042,	280.694 controle

left =  [[48	,23.5,	70,	2.707	,0.467408666666667,	8.8071,	9.7024,	7.99585	,417.114]]
right = [[73,	37.109375,	134,	5.636,	1.86288586666667,	41.4064	,3.335665,	6.89235,	788.902 ]]

print(left)
print(right)

print("\n\n\n")
print("------------------------------------")

left = scaler.transform(left)
right = scaler.transform(right)
print(left)
print(right)

print("\n\n\n")
print("------------------------------------")


left = left.reshape(1,9,1)
right = right.reshape(1,9,1)
print(left)
print(right)

print("\n\n\n")
print("------------------------------------")

test = [left, right]
print(test)

predict(model, test)